## Regression part b

### Imports and Data Loading

In [39]:
from ucimlrepo import fetch_ucirepo 
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import scipy.stats as st
import torch
# fetch dataset 
energy_efficiency = fetch_ucirepo(id=242) 
  
# data (as pandas dataframes) 
X = energy_efficiency.data.features 
y = energy_efficiency.data.targets 
N, M = X.shape

# Changing column names
X.columns = ['Relative Compactness', 'Surface Area', 'Wall Area', 'Roof Area', 'Overall Height', 'Orientation', 'Glazing Area', 'Glazing Area Distribution']
y.columns = ['Heating Load', 'Cooling Load']

# One-hot encoding
orientation_encoding = pd.get_dummies(X['Orientation'], prefix='Orientation', prefix_sep='=')
glazing_area_encoding = pd.get_dummies(X['Glazing Area Distribution'], prefix='Glazing_Area_Distribution', prefix_sep='=')
X = pd.concat([X.drop('Orientation', axis=1), orientation_encoding], axis=1)
X = pd.concat([X.drop('Glazing Area Distribution', axis=1), glazing_area_encoding], axis=1)
y1 = y['Heating Load'].values
y2 = y['Cooling Load'].values

X = X.astype(np.float32)
y = y.astype(np.float32)

In [ ]:
def setup_storage_for_experiment(K_outer, K_inner, num_lambdas, num_units):
    # Setup storage for the optimal hyperparameters found from the inner CV
    optimal_hyperparameters = {
        'lambda': np.empty(K_outer),
        'hidden_units': np.empty(K_outer)
    }

    # Setup storage for model coefficients and errors for each experiment in all inner folds
    # ws_inner = np.empty((M + 1, K_outer, K_inner, num_hyperparams))
    train_errors_inner = {
        'linear': np.empty((K_outer, K_inner, num_lambdas)),
        'ann': np.empty((K_outer, K_inner, num_units))
    }
    test_errors_inner = {
        'linear': np.empty((K_outer, K_inner, num_lambdas)),
        'ann': np.empty((K_outer, K_inner, num_units))
    }

    # Setup storage for model coefficients for each experiment in all outer folds
    # ws_outer = {
    #     'not regularized': np.empty((M + 1, K_outer)),
    #     'regularized': np.empty((M + 1, K_outer))
    # }
    # Setup storage for errors as a dictionary
    errors_outer = {
        'train': {
            'baseline': np.empty((K_outer, 1)), 
            'linear': np.empty((K_outer, 1)),
            'ann': np.empty((K_outer, 1))
        },
        'test': {
            'baseline': np.empty((K_outer, 1)), 
            'linear': np.empty((K_outer, 1)),
            'ann': np.empty((K_outer, 1))
        }
    }
    return optimal_hyperparameters, train_errors_inner, test_errors_inner, errors_outer

In [44]:
class ANNRegressor(torch.nn.Module):
    def __init__(self, num_features, hidden_units, num_outputs=2):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(num_features, hidden_units),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_units, num_outputs)
        )

    def forward(self, x):
        return self.net(x)


In [59]:
def train_ann(X_train, y_train, X_val, y_val, num_units, epochs=500, lr=0.01):
    model = ANNRegressor(X_train.shape[1], num_units, y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    loss_fn = torch.nn.MSELoss()

    X_train = torch.tensor(X_train.values, dtype=torch.float32)
    y_train = torch.tensor(y_train.values, dtype=torch.float32)
    X_val = torch.tensor(X_val.values, dtype=torch.float32)
    y_val = torch.tensor(y_val.values, dtype=torch.float32)

    for _ in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(X_train)
        loss = loss_fn(preds, y_train)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_preds = model(X_val).numpy()
    return model, val_preds


In [57]:
def multioutput_mse(y_true, y_pred):
    # y_true and y_pred shape: (N, 2)
    return np.mean(np.sum((y_true - y_pred)**2, axis=1))

In [69]:
K_outer = 10
K_inner = 10
seed = 42

CV_outer = KFold(K_outer, shuffle=True, random_state=seed)
CV_inner = KFold(K_inner, shuffle=True, random_state=seed)

all_true = []
all_pred_ann = []
all_pred_linear = []
all_pred_baseline = []

# lambdas = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3,]
lambdas = np.logspace(-2, 2, 50)
hidden_units = [1, 5, 10, 20, 25, 50]
numerical_features = ['Relative Compactness', 'Surface Area', 'Wall Area', 'Roof Area', 'Overall Height', 'Glazing Area']

optimal_hyperparameters, train_errors_inner, test_errors_inner, errors_outer = setup_storage_for_experiment(K_outer, K_inner, len(lambdas), len(hidden_units))

for outer_fold_idx, (outer_train_index, outer_test_index) in enumerate(CV_outer.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[outer_train_index, :], X.iloc[outer_test_index, :]
    y_train_outer, y_test_outer = y.iloc[outer_train_index], y.iloc[outer_test_index]
    
    
    for inner_fold_idx, (inner_train_index, inner_test_index) in enumerate(CV_inner.split(X_train_outer, y_train_outer)):
        X_train_inner, X_test_inner = X_train_outer.iloc[inner_train_index, :], X_train_outer.iloc[inner_test_index, :]
        y_train_inner, y_test_inner = y_train_outer.iloc[inner_train_index], y_train_outer.iloc[inner_test_index]
        
        # Calculate the mean and std of inner training loop and standardize datasets
        mu = np.mean(X_train_inner[numerical_features], axis=0)
        sigma = np.std(X_train_inner[numerical_features], axis=0)
            
        X_train_inner[numerical_features] = (X_train_inner[numerical_features] - mu) / sigma
        X_test_inner[numerical_features] = (X_test_inner[numerical_features] - mu) / sigma
        
        # Iterate over lambdas and calculate test errors
        for lambda_idx, regularization_strength in enumerate(lambdas):
            linear_model = Ridge(regularization_strength)
            linear_model.fit(X_train_inner, y_train_inner)
            
            train_errors_inner['linear'][outer_fold_idx, inner_fold_idx, lambda_idx] = multioutput_mse(y_train_inner, linear_model.predict(X_train_inner))
            test_errors_inner['linear'][outer_fold_idx, inner_fold_idx, lambda_idx] = multioutput_mse(y_test_inner, linear_model.predict(X_test_inner))
            
        # Iterate over number of hidden units and calculate test errors
        for unit_idx, num_units in enumerate(hidden_units):
            model, val_pred = train_ann(X_train_inner, y_train_inner, X_test_inner, y_test_inner, num_units=num_units)
            test_errors_inner['ann'][outer_fold_idx, inner_fold_idx, unit_idx] = multioutput_mse(y_test_inner.values, val_pred)

    # Choose optimal hyperparameters for each model
    optimal_lambda = lambdas[np.argmin(np.mean(test_errors_inner['linear'][outer_fold_idx], axis=0))]
    optimal_units = hidden_units[np.argmin(np.mean(test_errors_inner['ann'][outer_fold_idx], axis=0))]
    
    # Store the optimal regularization strength for the current outer fold
    optimal_hyperparameters['lambda'][outer_fold_idx] = optimal_lambda
    optimal_hyperparameters['hidden_units'][outer_fold_idx] = optimal_units

    # Compute the mean and standard deviation of the outer training data split, then standardize the training and test sets

    mu_outer = np.mean(X_train_outer[numerical_features], axis=0)
    sigma_outer = np.std(X_train_outer[numerical_features], axis=0)
    
    X_train_outer[numerical_features] = (X_train_outer[numerical_features] - mu_outer) / sigma_outer
    X_test_outer[numerical_features] = (X_test_outer[numerical_features] - mu_outer) / sigma_outer
    
    # Create and fit models with optimal hyperparameters
    linear_model = Ridge(optimal_lambda)
    linear_model.fit(X_train_outer, y_train_outer)
    
    errors_outer['train']['linear'][outer_fold_idx] = multioutput_mse(y_train_outer, linear_model.predict(X_train_outer))
    errors_outer['test']['linear'][outer_fold_idx] = multioutput_mse(y_test_outer, linear_model.predict(X_test_outer))
    all_pred_linear.append(linear_model.predict(X_test_outer))
    
    ann_model, test_pred = train_ann(X_train_outer, y_train_outer, X_test_outer, y_test_outer, num_units=optimal_units)
    
    errors_outer['test']['ann'][outer_fold_idx] = multioutput_mse(y_test_outer, test_pred)
    all_pred_ann.append(test_pred)
    
    # Baseline: predict mean of y_train_outer
    baseline_train_mean = y_train_outer.mean(axis=0)
    baseline_pred_train = np.tile(baseline_train_mean, (len(y_train_outer), 1))
    baseline_pred_test = np.tile(baseline_train_mean, (len(y_test_outer), 1))
    all_pred_baseline.append(baseline_pred_test)
    
    errors_outer['train']['baseline'][outer_fold_idx] = multioutput_mse(y_train_outer, baseline_pred_train)
    errors_outer['test']['baseline'][outer_fold_idx] = multioutput_mse(y_test_outer, baseline_pred_test)
    
    all_true.append(y_test_outer.values)

table_data = {
    "Outer fold i": np.arange(1, K_outer+1),
    "h* (ANN)": optimal_hyperparameters['hidden_units'],
    "E_test (ANN)": np.round(errors_outer['test']['ann'].flatten(), 4),
    "λ* (Linear)": np.round(optimal_hyperparameters['lambda'], 2),
    "E_test (Linear)": np.round(errors_outer['test']['linear'].flatten(), 4),
    "E_test (Baseline)": np.round(errors_outer['test']['baseline'].flatten(), 4)
}

results_table = pd.DataFrame(table_data)

print("\nTwo-level Cross-Validation Results Table:\n")
print(results_table.to_string(index=False))
            
        
        

/tmp/ipykernel_11213/2517860200.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_inner[numerical_features] = (X_train_inner[numerical_features] - mu) / sigma
/tmp/ipykernel_11213/2517860200.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_inner[numerical_features] = (X_test_inner[numerical_features] - mu) / sigma
/tmp/ipykernel_11213/2517860200.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v


Two-level Cross-Validation Results Table:

 Outer fold i  h* (ANN)  E_test (ANN)  λ* (Linear)  E_test (Linear)  E_test (Baseline)
            1      20.0       20.5975         0.24          20.9220           210.8831
            2      50.0       13.1610         0.17          14.5031           185.1296
            3      50.0       14.8641         0.24          19.9435           181.8142
            4      50.0        9.5922         0.29          16.1423           196.2293
            5      50.0        5.9965         0.24          14.5296           202.6197
            6      25.0       26.8640         0.20          26.3629           207.4743
            7      50.0       18.2485         0.29          17.5428           199.4795
            8      50.0       10.9722         0.24          18.4718           192.9943
            9      50.0       17.2170         0.36          19.7396           190.3836
           10      50.0       13.0066         0.24          14.3135           156.8149

### Confidence interval calculation

In [67]:
l1_loss = lambda y, y_pred: np.abs(y - y_pred)
l2_loss = lambda y, y_pred: (y - y_pred)**2

def confidence_interval_comparison(y_true, y_preds_A, y_preds_B, loss_fn, alpha=0.05):

    # Calculate estimated error, z_hat, as the mean loss across all samples
    n_outputs = y_true.shape[1]
    results = {
        'z_hat' : np.empty(n_outputs),
        'CI': np.empty((n_outputs, 2)),
        'p_value':np.empty(n_outputs)
    }
    
    for j in range(n_outputs):
        z_A = loss_fn(y_true[:, j], y_preds_A[:, j])
        z_B = loss_fn(y_true[:, j], y_preds_B[:, j])
        z = z_A - z_B
        z_hat = np.mean(z)
    
        n = len(y_true)
        nu = n - 1  # degrees of freedom

        sem = np.sqrt(sum(((z - z_hat)**2) / (n * nu)))
    
        CI = st.t.interval(1 - alpha, df=nu, loc=z_hat, scale=sem)

        t_stat = -np.abs(np.mean(z)) / st.sem(z)
        p_value = 2 * st.t.cdf(t_stat, df=nu)
        
        results['z_hat'][j] = z_hat
        results['CI'][j] = np.array(CI)
        results['p_value'][j] = p_value

    return results

alpha = 0.05

y_true = np.vstack(all_true)          #
y_linear = np.vstack(all_pred_linear)
y_ann = np.vstack(all_pred_ann)
y_baseline = np.vstack(all_pred_baseline)

results_linear_vs_ann = confidence_interval_comparison(y_true, y_linear, y_ann, loss_fn=l2_loss)
results_linear_vs_baseline = confidence_interval_comparison(y_true, y_linear, y_baseline, loss_fn=l2_loss)
results_ann_vs_baseline = confidence_interval_comparison(y_true, y_ann, y_baseline, loss_fn=l2_loss)

print("Difference between linear regression and ANN")
for i in range (2):
    if (i == 0):
        print("Heating", end=" -> ")
    else:
        print("Cooling", end=" -> ")
        
    print (f"CI: ({results_linear_vs_ann['CI'][i][0]}, {results_linear_vs_ann['CI'][i][1]}), p-value = {results_linear_vs_ann['p_value'][i]}")
    
print("\nDifference between linear regression and baseline")
for i in range (2):
    if (i == 0):
        print("Heating", end=" -> ")
    else:
        print("Cooling", end=" -> ")
        
    print (f"CI: ({results_linear_vs_baseline['CI'][i][0]}, {results_linear_vs_baseline['CI'][i][1]}), p-value = {results_linear_vs_baseline['p_value'][i]}")

print("\nDifference between ANN and baseline")
for i in range (2):
    if (i == 0):
        print("Heating", end=" -> ")
    else:
        print("Cooling", end=" -> ")
        
    print (f"CI: ({results_ann_vs_baseline['CI'][i][0]}, {results_ann_vs_baseline['CI'][i][1]}), p-value = {results_ann_vs_baseline['p_value'][i]}")


Difference between linear regression and ANN
Heating -> CI: (1.7109650363863922, 2.7022558460293795), p-value = 1.4608993176741286e-17
Cooling -> CI: (1.5572122312250145, 2.954063537055015), p-value = 3.9220909357241353e-10

Difference between linear regression and baseline
Heating -> CI: (-99.88567747968017, -87.82262024981202), p-value = 1.0543515480502158e-134
Cooling -> CI: (-85.59841290152004, -75.05316790902684), p-value = 7.278320441522245e-131

Difference between ANN and baseline
Heating -> CI: (-102.27761510397853, -89.84390589211522), p-value = 2.0162191800083546e-133
Cooling -> CI: (-88.14095879756506, -77.021898258099), p-value = 2.2359502971618583e-126
